In [ ]:
import numpy as np
import cv2
import os
import re
from scipy import ndimage
from tqdm import tqdm
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Reshape, UpSampling2D

from google.colab.patches import cv2_imshow
from google.colab import drive

In [ ]:
!mkdir /content/images

drive.mount('/content/drive')

!unzip -q '/content/drive/My Drive/NATO/trajectory_images.zip' -d '/content/images'

In [ ]:
Dict = {0: 'advance_to_contact', 2: 'attack', 9: 'counterattack', 18: 'main_attack'}
Dict_images = {key: [] for key in Dict.values()}

In [ ]:
def cut_excess_white(symbol, excess_str = 120):
    symbol = symbol[np.argwhere(np.amin(symbol,axis=1) < excess_str)[0][0]:np.argwhere(np.amin(symbol,axis=1) < excess_str)[-1][0],:]
    symbol = symbol[:,np.argwhere(np.amin(symbol,axis=0) < excess_str)[0][0]:np.argwhere(np.amin(symbol,axis=0) < excess_str)[-1][0]]
    return symbol

In [ ]:
data = '/content/images'
file_list=os.listdir(data)

symbols_regex = '([a-zA-Z_ ]*)\d*.*'
for filename in tqdm(file_list):
    img = cv2.imread(data+"/"+filename, 0)
    img[img <= 100] = 0
    img[img > 100] = 255
    img = cut_excess_white(img)
    key = re.findall(symbols_regex, filename)[0]
    Dict_images[key].append(img)

In [ ]:
Dict_images_all = {key: [] for key in Dict.values()}

In [ ]:
def rotate_images(image, symbol):
    for degree in range(0, 359, 5):
        rotation_angle = degree
        img = ndimage.rotate(image, rotation_angle, mode='constant', cval=255)
        dim = (120, 120)
        resized_img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        Dict_images_all[symbol].append(resized_img)

In [ ]:
for i in Dict_images.keys():
    print(str(i))
    for img in tqdm(Dict_images[str(i)]):
        rotate_images(img, str(i))

In [ ]:
def model_version1():
    # autoencoder model
    # encoder
    input = Input(shape=(120, 120, 1))
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input)
    x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D(pool_size=(2, 2), padding='same', name="encoded")(x)
    #decoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    model = Model(inputs=input, outputs=decoded)

    return model

In [ ]:
model = model_version1()

for i in Dict_images_all.keys():
    str(i)
    model.load_weights('/content/drive/MyDrive/models_trajectory/'+str(i)+'_trajectory_autoencoder.h5')
    encoder = Model(inputs=model.input, outputs=model.get_layer("encoded").output)

    images = np.array(Dict_images_all[str(i)])
    images = images.astype('float32') / 255.
    images = images.reshape(images.shape[0], images.shape[1], images.shape[2], 1)

    features = encoder.predict(images)
    data = {"images": Dict_images_all[str(i)], "features": features}
    np.save(str(i)+"_trajectory_data.npy", data)